# Web scraping a Wikipidea page
This project is divided into two sections:
1. We will extract the information about the largest companies by revenue in the US from a Wikipedia article.
2. We will place the information in a pandas dataset and perform some cleaning

## Extracting the information and building the pandas DataFrame

In [212]:
#Importing the necessary libraries
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [213]:
#The URL to the Wikipedia article
url = "https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue"
#Obtaining the html code of the site as a text
page = requests.get(url).text
#Using BeautifulSoup to work with this text
soup = BeautifulSoup(page, 'html')

In [214]:
#Finding the section that contains the table that we want to work on
#There are multiple tables in this site, the one that we are interested in is the first shown in the site (second in the html code)
table = soup.find_all('table')[1]

In [215]:
#Getting the table headers
titles = table.find_all('th')
print(titles)

[<th>Rank
</th>, <th>Name
</th>, <th>Industry
</th>, <th>Revenue <br/>(USD millions)
</th>, <th>Revenue growth
</th>, <th>Employees
</th>, <th>Headquarters
</th>]


In [216]:
#Cleaning the headers
titles_clean = [title.text.strip() for title in titles]
print(titles_clean)

['Rank', 'Name', 'Industry', 'Revenue (USD millions)', 'Revenue growth', 'Employees', 'Headquarters']


In [217]:
#Now we will begin building the pandas dataframe with this headers
df = pd.DataFrame(columns= titles_clean)
df

,Rank,Name,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters


In [218]:
#Now we will extract the information in each row
raw_rows = [raw_row.text.split('\n') for raw_row in table.find_all('tr')[1:]]
#Information in the first row extracted in the previous line of code
print(raw_rows[0])

['', '1', '', 'Walmart', '', 'Retail', '', '611,289', '', '  6.7%', '', '2,100,000', '', 'Bentonville, Arkansas', '']


In [219]:
#Cleaning the rows by removing Null values and blank spaces
clean_rows = []
for row in raw_rows:
  new_row = [value.strip() for value in row if value !='']
  clean_rows.append(new_row)
#showing the first row after cleaning
print(clean_rows[0])

['1', 'Walmart', 'Retail', '611,289', '6.7%', '2,100,000', 'Bentonville, Arkansas']


In [220]:
#Inserting the rows into the dataframe
for index in range(len(clean_rows)):
  df.loc[index] = clean_rows[index]

In [221]:
#Now we have the exact same table found in the url.
df.head()

,Rank,Name,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters
0,1,Walmart,Retail,"611,289",6.7%,"2,100,000","Bentonville, Arkansas"
1,2,Amazon,Retail and cloud computing,"513,983",9.4%,"1,540,000","Seattle, Washington"
2,3,ExxonMobil,Petroleum industry,"413,680",44.8%,"62,000","Spring, Texas"
3,4,Apple,Electronics industry,"394,328",7.8%,"164,000","Cupertino, California"
4,5,UnitedHealth Group,Healthcare,"324,162",12.7%,"400,000","Minnetonka, Minnesota"


## Data cleaning
* All the values in the dataframe are strings, so we want to transform them into the proper types.
* Because the 'Headquarters' column contains cities and states, we will separate it into two columns.


In [222]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Rank                    100 non-null    object
 1   Name                    100 non-null    object
 2   Industry                100 non-null    object
 3   Revenue (USD millions)  100 non-null    object
 4   Revenue growth          100 non-null    object
 5   Employees               100 non-null    object
 6   Headquarters            100 non-null    object
dtypes: object(7)
memory usage: 6.2+ KB
None


In [223]:
#We will begin by making the 'Rank' column numeric
df['Rank'] = pd.to_numeric(df['Rank'])

In [224]:
df['Rank'].dtype

dtype('int64')

In [225]:
#Next, we will split the 'Headquarters' column into 'hq_city' and 'hq_state'
#Extracting the city and state
city = []
state = []
for value in df['Headquarters']:
  city.append(value.split(',')[0])
  state.append(value.split(',')[1])
# creating the columns
df['hq_city'] = city
df['hq_state'] = state
df.drop('Headquarters', axis=1, inplace=True)

In [226]:
df.head()

,Rank,Name,Industry,Revenue (USD millions),Revenue growth,Employees,hq_city,hq_state
0,1,Walmart,Retail,"611,289",6.7%,"2,100,000",Bentonville,Arkansas
1,2,Amazon,Retail and cloud computing,"513,983",9.4%,"1,540,000",Seattle,Washington
2,3,ExxonMobil,Petroleum industry,"413,680",44.8%,"62,000",Spring,Texas
3,4,Apple,Electronics industry,"394,328",7.8%,"164,000",Cupertino,California
4,5,UnitedHealth Group,Healthcare,"324,162",12.7%,"400,000",Minnetonka,Minnesota


In [227]:
#To convert the Employees column into a numeric type, we first must deal with a particular value that stops us from turning all the values into numbers
#Employees value 60 originally included a hyperlink
df[df['Employees'] == '4,000[2]']

,Rank,Name,Industry,Revenue (USD millions),Revenue growth,Employees,hq_city,hq_state
60,61,StoneX Group,Financials,"66,036",55.3%,"4,000[2]",New York City,New York


In [228]:
#Manually removing the bracketed number
df['Employees'][df['Employees'] == '4,000[2]'] = '4,000'

<ipython-input-228-db0f659b4786>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Employees'][df['Employees'] == '4,000[2]'] = '4,000'


In [229]:
print(df.iloc[60])

Rank                                 61
Name                       StoneX Group
Industry                     Financials
Revenue (USD millions)           66,036
Revenue growth                    55.3%
Employees                         4,000
hq_city                   New York City
hq_state                       New York
Name: 60, dtype: object


In [230]:
# Now we can transform the values in the Employees column to numbers
df['Employees'] = [int(value) for value in df['Employees'].str.replace(',', '')]

In [231]:
#Employees is now numerical
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Rank                    100 non-null    int64 
 1   Name                    100 non-null    object
 2   Industry                100 non-null    object
 3   Revenue (USD millions)  100 non-null    object
 4   Revenue growth          100 non-null    object
 5   Employees               100 non-null    int64 
 6   hq_city                 100 non-null    object
 7   hq_state                100 non-null    object
dtypes: int64(2), object(6)
memory usage: 7.0+ KB


In [232]:
#We will now turn the 'Revenue growth' column into a numerical column and rename it to clarify that its values are percentages
df['Revenue growth'] = [float(value) for value in df['Revenue growth'].str.replace('%','')]
df.rename(columns={'Revenue growth':'Revenue growth (%)'},inplace=True)

In [233]:
#Finally, we will make the 'Revenue (USD millions)' column numerical
df['Revenue (USD millions)'] = [float(value) for value in df['Revenue (USD millions)'].str.replace(',','.')]

In [234]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Rank                    100 non-null    int64  
 1   Name                    100 non-null    object 
 2   Industry                100 non-null    object 
 3   Revenue (USD millions)  100 non-null    float64
 4   Revenue growth (%)      100 non-null    float64
 5   Employees               100 non-null    int64  
 6   hq_city                 100 non-null    object 
 7   hq_state                100 non-null    object 
dtypes: float64(2), int64(2), object(4)
memory usage: 7.0+ KB
None


In [235]:
#We will now export the table as a csv
df.to_csv('drive/MyDrive/ML Projects/Web scraping project/Companies-by-Revenue.csv',index=False)